<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC Dashboard
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas%20Dashboard/Naas_Dashboard_Financial_Consolidation.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Naas+Dashboard+-+Financial+Consolidation:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #naasdashboard #plotly #dash #naas #asset #automation #analytics #snippet #datavizualisation

**Author:** [Meriem Si](https://www.linkedin.com/in/meriem-si-104236181/)

**Description:** This notebook provides a comprehensive dashboard for financial consolidation and analysis.

## Input

### Import libraries

In [1]:
try:
    import dash
except:
    !pip install dash --user
    import dash
from dash import html, dcc, Input, Output, State
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
from naas_drivers import gsheet
from dash_bootstrap_components._components.Container import Container
from plotly.subplots import make_subplots

### Defining the port of the dashboard

In [2]:
DASH_PORT = 8050

### Setup Google Sheets

In [3]:
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/127PqDiuz5CXa8tTzhhgUl_ziMYThwZhfWOi86Bo-h-g/edit?usp=sharing"
SHEET_NAME = "KPIS"
SHEET_NAME_2 = "REVENUE"
SHEET_NAME_3 = "OPERATIONAL_EXPENSES"
SHEET_NAME_4 = "EARNING_BEFORE_INTREST_AND_TAXES"
SHEET_NAME_5 = "INCOME_STATEMENT"

### Setup Dash App

In [4]:
APP_TITLE = "FEC Dashboard"
APP_LOGO = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRszy3bMYjE7-_YpiDDqWrLEGR-wJKEbLyff-VQMJOZoA&s"

In [5]:
# Style used for card
CARD_COL_STYLE1 = {"className": "g-2", "xs": 12, "sm": 12, "md": 12, "lg": 2, "xl": 2}

CARD_COL_STYLE2 = {
    #     "className": "gx-5",
    "xs": 12,
    "sm": 12,
    "md": 12,
    "lg": 12,
    "xl": 12,
}

CARD_COL_STYLE3 = {
    #     "className": "gx-5",
    "xs": 12,
    "sm": 12,
    "md": 12,
    "lg": 6,
    "xl": 6,
}

In [6]:
# Color used for card
CARD_COLOR = ["#fbfbfb", "#fbfbfb", "#fbfbfb", "#fbfbfb", "#fbfbfb", "#fbfbfb"]
BACKGROUND_COLOR = "#f8f9fa"

## Model

### Data

#### Dropdown data: Ref Entities / Ref Scenarios

In [7]:
# Lists used for dropdowns intractiveness
entities = ["Entity1", "Entity2"]

scenarios = ["2022", "2021", "2020"]

#### Get Kpis data

In [8]:
df_hkpis = gsheet.connect(SPREADSHEET_URL).get(SHEET_NAME)
df_hkpis["VALUE_D"] = (
    df_hkpis["VALUE"].astype(str) + " " + df_hkpis["UNITS"].astype(str)
)
print("Nb row:", len(df_hkpis))
df_hkpis.head(6)

Nb row: 36


,ENTITY,SCENARIO,LABEL,VALUE,UNITS,VALUE_D
0,Entity1,2022,Revenue Target,80,%,80 %
1,Entity1,2022,COGS Ratio,20,%,20 %
2,Entity1,2022,Gross Profit Margin,84,%,84 %
3,Entity1,2022,Opex Ratio,74,%,74 %
4,Entity1,2022,Ebit Margin,18,%,18 %
5,Entity1,2022,Net Profit Margin,14,%,14 %


#### Get Revenue data

In [9]:
df_revenue = gsheet.connect(SPREADSHEET_URL).get(SHEET_NAME_2)
print("Nb row:", len(df_revenue))
df_revenue.head(6)

Nb row: 72


,ENTITY,SCENARIO,DATE,VALUE_BAR,VAlUE_LINE1,VAlUE_LINE2
0,Entity1,2022,Jan 15,45,89,78
1,Entity1,2022,Fev 15,58,91,71
2,Entity1,2022,Mar 15,21,93,72
3,Entity1,2022,Apl 15,28,92,73
4,Entity1,2022,mai 15,24,90,70
5,Entity1,2022,Jun 15,51,88,78


#### Get Operational expenses data

In [10]:
df_operational_expenses = gsheet.connect(SPREADSHEET_URL).get(SHEET_NAME_3)
print("Nb row:", len(df_operational_expenses))
df_operational_expenses.head(6)

Nb row: 288


,ENTITY,SCENARIO,DATE,VALUE,COUNT
0,Entity1,2022,Jan 15,General,10
1,Entity1,2022,Jan 15,Marketing,32
2,Entity1,2022,Jan 15,Sales,20
3,Entity1,2022,Jan 15,IT,55
4,Entity1,2022,Feb 15,General,25
5,Entity1,2022,Feb 15,Marketing,65


#### Get Earning before intrest and taxes data

In [11]:
df_earningBIT = gsheet.connect(SPREADSHEET_URL).get(SHEET_NAME_4)
print("Nb row:", len(df_earningBIT))
df_earningBIT.head(6)

Nb row: 72


,ENTITY,SCENARIO,DATE,VALUE1,VALUE2
0,Entity1,2022,Jan 15,55,44
1,Entity1,2022,Fev 15,68,42
2,Entity1,2022,Mar 15,51,43
3,Entity1,2022,Apl 15,42,18
4,Entity1,2022,mai 15,85,16
5,Entity1,2022,Jun 15,25,42


#### Get Income Statement data

In [12]:
df_income_statement = gsheet.connect(SPREADSHEET_URL).get(SHEET_NAME_5)
df_income_statement["VALUE_D"] = (
    df_income_statement["VALUE"].astype(str)
    + " "
    + df_income_statement["UNITS"].astype(str)
)
print("Nb row:", len(df_income_statement))
df_income_statement.head(6)

Nb row: 78


,ENTITY,SCENARIO,LABEL,VALUE,UNITS,VALUE_D
0,Entity1,2022,Revenue,"1,236,544",€,"1,236,544 €"
1,Entity1,2022,COGS,"200,324",€,"200,324 €"
2,Entity1,2022,Gross Profit,"210,324",€,"210,324 €"
3,Entity1,2022,Opex,"60,324",€,"60,324 €"
4,Entity1,2022,Sales,"660,324",€,"660,324 €"
5,Entity1,2022,Marketing,"50,324",€,"50,324 €"


### Charts

#### Chart 1 : Revenue

In [13]:
def create_barlinechart(
    df,
    label="DATE",
    value_bar="VALUE_BAR",
    value_scatter="VAlUE_LINE1",
    value_scatter_dot="VAlUE_LINE2",
    xaxis_title=None,
    yaxis_title_r=None,
    yaxis_title_l=None,
):
    fig = go.Figure()
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(
            x=df[label],
            y=df[value_bar],
            name="Growth",
            marker=dict(color="#ADD8E6"),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=df[label],
            y=df[value_scatter],
            mode="lines",
            name="Revenue",
            line=dict(color="royalblue", width=2.5),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=df[label],
            y=df[value_scatter_dot],
            name="Target",
            line=dict(color="green", width=4, dash="dot"),
        ),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        legend=dict(orientation="h"),
        plot_bgcolor="white",
        height=300,
        paper_bgcolor="white",
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        margin=dict(l=20, r=20, t=20, b=20),
    )

    fig.update_layout(
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig.show()
    return fig


# fig = create_barlinechart(df_revenue_test,
#                           yaxis_title_r=None,
#                           yaxis_title_l=None)

#### Chart 2 : Operational Expenses

In [14]:
def stacked_bar(df, labels, values):
    fig2 = px.bar(
        df,
        x="DATE",
        y="COUNT",
        color="VALUE",
        #                   width=520,
        height=300,
        color_discrete_map={
            "General": "#5ac73c",
            "Marketing": "#6dabc7",
            "Sales": "#307a37",
            "IT": "#30337a",
        },
    )

    fig2.update_traces(width=0.5)

    fig2.update_layout(
        yaxis_tickformat=",",
        legend_title="",
        legend=dict(orientation="h"),
        plot_bgcolor="white",
        paper_bgcolor="white",
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        margin=dict(l=20, r=20, t=20, b=20),
    )

    fig2.update_layout(
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    # Set the visibility OFF
    fig2.update_layout(
        uniformtext_minsize=8,
        uniformtext_mode="hide",
        xaxis_title=None,
        yaxis_title=None,
    )

    return fig2


# stacked_bar = stacked_bar(df2, None, None)

#### Chart 3 : Earnings before Intrest and Taxes

In [15]:
def create_doublelinechart(
    df, label="DATE", value1="VALUE1", value2="VALUE2", xaxis_title="Months"
):
    fig3 = go.Figure()
    # Add traces

    fig3.add_trace(
        go.Scatter(
            x=df[label],
            y=df[value1],
            mode="lines",
            name="EBIT Actual",
            line=dict(color="#0A66C2", width=2.5),
        ),
    )

    fig3.add_trace(
        go.Scatter(
            x=df[label],
            y=df[value2],
            mode="lines",
            name="EBIT Target",
            line=dict(color="green", width=4, dash="dot"),
        ),
    )

    # Add figure title
    fig3.update_layout(
        legend=dict(orientation="h"),
        plot_bgcolor="white",
        height=300,
        #         width=520,
        paper_bgcolor="white",
        xaxis_title=xaxis_title,
        xaxis_title_font=dict(family="Arial", size=10, color="black"),
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        margin=dict(l=20, r=20, t=20, b=40),
    )

    # Set the visibility OFF
    fig3.update_layout(
        uniformtext_minsize=8,
        uniformtext_mode="hide",
        xaxis_title=None,
        yaxis_title=None,
    )

    fig3.update_layout(
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig3.show()
    return fig3


# doubleline = create_doublelinechart(df3)

### Design App

#### Create Dropdown

In [16]:
# Entity's dropdown list
dropdown_entity = dcc.Dropdown(
    id="entity",
    options=[{"label": i, "value": i} for i in entities],
    placeholder="Entity",
    value=entities[0],
)

# Scenario's dropdown list
dropdown_scenario = dcc.Dropdown(
    id="scenario",
    options=[{"label": i, "value": i} for i in scenarios],
    placeholder="Scenario",
    value=scenarios[0],
)

#### Create Navbar

In [17]:
navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src=APP_LOGO, height="50")),
                        dbc.Col(dbc.NavbarBrand(APP_TITLE, className="ms-2")),
                    ],
                    align="center",
                    className="g-0",
                ),
            ),
            dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
            dbc.Collapse(
                dbc.Nav(
                    [
                        html.Div(
                            [
                                html.Div(className="w-100"),
                                html.Div(className="w-100"),
                                html.Div(dropdown_entity, className="w-100"),
                                html.Div(dropdown_scenario, className="w-100"),
                            ],
                            className="pt-1 pb-1 d-grid gap-2 d-md-flex w-100",
                        )
                    ],
                    className="ms-auto w-100",
                    navbar=True,
                ),
                id="navbar-collapse",
                navbar=True,
                is_open=False,
            ),
        ],
    ),
    color="#181A1C",
    dark=True,
)

#### Create Card

In [18]:
# Function to create card
def create_card_col(uid, card_title, card_paragraph, card_paragraphvar, card_color):
    card = dbc.Col(
        dbc.Card(
            dbc.CardBody(
                [
                    html.P(
                        card_title,
                        className="card-title",
                        style={
                            "font-size": "15px",
                            "padding-bottom": "5px",
                            "text-align": "center",
                            "font-weight": "bold",
                            "color": "#181A1C",
                        },
                    ),
                    html.P(
                        card_paragraph,
                        className="card-text",
                        style={"font-size": "35px", "text-align": "center","font-weight": "bold","color": "#181A1C"},
                        id=uid,
                    ),
                    
                    html.P(
                        card_paragraphvar,
                        className="card-text",
                        style={"font-size": "16px", "text-align": "center","color": "#181A1C"},
                        id=uid,
                    ),
                    
                ]
            ),
            color=card_color,
            inverse=True,
        ),
        **CARD_COL_STYLE1,
    )
    return card

#### Create Chart Card

In [19]:
def create_chart(title, chart_id, CARD_COL_STYLE):
    card = dbc.Col(
        dbc.Card(
            dbc.CardBody(
                [
                    html.H5(
                        title,
                        style={
                            "padding-top": "20px",
                            "margin-left": "20px",
                            "font-weight": "bold",
                            "font-size": "17px",
                        },
                    ),
                    dcc.Graph(
                        id=chart_id, config={"displayModeBar": True, "staticPlot": True}
                    ),
                ],
            ),
            color=CARD_COLOR,
            inverse=False,
        ),
        style={
            "background-color": BACKGROUND_COLOR,
            "display": "inline",
#             "padding": "0px",
        },
        **CARD_COL_STYLE
    )
    return card

#### Create app layout

In [20]:
app = dash.Dash(
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

# app = dash.Dash() if you are not in Naas
app.title = APP_TITLE
app.layout = html.Div(
    [
        # Navbar
        navbar,
        # HKPIS
        dbc.Row(
            [
                create_card_col("card1", "CHIFFRE D'AFFAIRES", "85%", "+10%", CARD_COLOR[0]),
                create_card_col("card2", "MARGE BRUTE", "78%", "+10%",CARD_COLOR[1]),
                create_card_col("card3", "EBE", "55%", "+10%",CARD_COLOR[2]),
                create_card_col("card4", "BFR", "85%", "+10%",CARD_COLOR[3]),
                create_card_col("card5", "CREANCES CLIENTS", "52%", "+10%",CARD_COLOR[4]),
                create_card_col("card6", "DETTES FOURNISSEURS", "95%", "+10%",CARD_COLOR[5]),
            ],
            
            style={
                "padding-left": "20px", 
                "padding-right": "20px"
            },
            className="g-0 d-flex align-items-center",
        ),
        # SPACE
        html.Br(),
        
        dbc.Row(
            [
                create_chart(
                    "OPERATIONAL EXPENSES", "fig2", CARD_COL_STYLE3
                ),
                create_chart(
                    "EARNINGS BEFORE INTREST AND TAXES",
                    "fig3",
                    CARD_COL_STYLE3,
                ),
            ],
            
            style={
                "padding-left": "20px", 
                "padding-right": "20px"
            },
        ),
        
        # SPACE
        html.Br(),
        
        dbc.Row(
            [
                create_chart(
                    "OPERATIONAL EXPENSES", "fig2", CARD_COL_STYLE3
                ),
                create_chart(
                    "EARNINGS BEFORE INTREST AND TAXES",
                    "fig3",
                    CARD_COL_STYLE3,
                ),
            ],
            
            style={
                "padding-left": "20px", 
                "padding-right": "20px"
            },
        ),
        
        
#         # CHARTS
#         dbc.Row(
#             [
#                 # Column 1: Charts
#                 dbc.Col(
#                     [
#                         # Row 1: 1 Charts
#                         dbc.Row(
#                             create_chart("REVENUE", "fig1", CARD_COL_STYLE2),
#                         ),
#                         # Row 2: 2 Charts
#                         dbc.Row(
#                             [
#                                 create_chart(
#                                     "OPERATIONAL EXPENSES", "fig2", CARD_COL_STYLE3
#                                 ),
#                                 create_chart(
#                                     "EARNINGS BEFORE INTREST AND TAXES",
#                                     "fig3",
#                                     CARD_COL_STYLE3,
#                                 ),
#                             ]
#                         ),
#                     ],
#                     xs=12,
#                     sm=12,
#                     md=12,
#                     lg=6,
#                     xl=6,
#                 ),
#             ],
            
#             style={
#                 "padding-left": "20px", 
#                 "padding-right": "20px"
#             },
#         ),
        
        # SPACE
        html.Br(),
    ],
)

# add callback for toggling the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


# add callback to filter data in cards and charts
@app.callback(
    [
        Output("card1", "children"),
        Output("card2", "children"),
        Output("card3", "children"),
        Output("card4", "children"),
        Output("card5", "children"),
        Output("card6", "children"),
        Output("fig1", "figure"),
        Output("fig2", "figure"),
        Output("fig3", "figure"),
    ],
    [Input("entity", "value"), Input("scenario", "value")],
)
def multi_outputs(entity, scenario):
    if entity is None and scenario is None:
        raise PreventUpdate

    # Get HKPIs dataframe
    dfhkpis = df_hkpis.copy()
    dfhkpis = dfhkpis[
        (dfhkpis["ENTITY"] == entity) & (dfhkpis["SCENARIO"].astype(str) == scenario)
    ].reset_index(drop=True)

    # Return card data
    card1 = dfhkpis.loc[dfhkpis["LABEL"] == "Revenue Target", "VALUE_D"].values[0]
    card2 = dfhkpis.loc[dfhkpis["LABEL"] == "COGS Ratio", "VALUE_D"].values[0]
    card3 = dfhkpis.loc[dfhkpis["LABEL"] == "Gross Profit Margin", "VALUE_D"].values[0]
    card4 = dfhkpis.loc[dfhkpis["LABEL"] == "Opex Ratio", "VALUE_D"].values[0]
    card5 = dfhkpis.loc[dfhkpis["LABEL"] == "Ebit Margin", "VALUE_D"].values[0]
    card6 = dfhkpis.loc[dfhkpis["LABEL"] == "Net Profit Margin", "VALUE_D"].values[0]

    # Get Revenue graph dataframe
    dfrevenue = df_revenue.copy()
    dfrevenue = dfrevenue[
        (dfrevenue["ENTITY"] == entity)
        & (dfrevenue["SCENARIO"].astype(str) == scenario)
    ].reset_index(drop=True)

    # Get Operational expenses graph dataframe
    dfoperationalexpenses = df_operational_expenses.copy()
    dfoperationalexpenses = dfoperationalexpenses[
        (dfoperationalexpenses["ENTITY"] == entity)
        & (dfoperationalexpenses["SCENARIO"].astype(str) == scenario)
    ].reset_index(drop=True)

    # Get Earning before intrest and taxes graph dataframe
    dfearningBIT = df_earningBIT.copy()
    dfearningBIT = dfearningBIT[
        (dfearningBIT["ENTITY"] == entity)
        & (dfearningBIT["SCENARIO"].astype(str) == scenario)
    ].reset_index(drop=True)

    # Create graphs
    fig1 = create_barlinechart(dfrevenue, yaxis_title_r=None, yaxis_title_l=None)
    fig2 = stacked_bar(dfoperationalexpenses, None, None)
    fig3 = create_doublelinechart(dfearningBIT)

    # Get Earning before intrest and taxes graph dataframe
    dfincome_statement = df_income_statement.copy()
    dfincome_statement = dfincome_statement[
        (dfincome_statement["ENTITY"] == entity)
        & (dfincome_statement["SCENARIO"].astype(str) == scenario)
    ].reset_index(drop=True)
    return (
        card1,
        card2,
        card3,
        card4,
        card5,
        card6,
        fig1,
        fig2,
        fig3,
    )

## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")

Dash is running on https://app.naas.ai/user/hello@alexandrestevens.fr/proxy/8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
[2023-05-24 11:28:18,786] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/home/ftp/.local/lib/python3.9/site-packages/flask/app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ftp/.local/lib/python3.9/site-packages/flask/app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ftp/.local/lib/python3.9/site-packages/flask/app.py", line 1482, in full_dispatch_request
    rv = self.preprocess_request()
  File "/home/ftp/.local/lib/python3.9/site-packages/flask/app.py", line 1974, in preprocess_request
    rv = self.ensure_sync(before_func)()
  File "/home/ftp/.local/lib/python3.9/site-packages/dash/dash.py", line 1316, in _setup_server
    _validate.validate_layout(self.layout, self._layout_value())
  File "/home/ftp/.local/lib/python3.9/site-packages/dash/_validate.py", line 416, in validate_layout
    raise exceptions.Dupl